<a href="https://colab.research.google.com/github/Fhupsel/Parent_RAG/blob/main/ParentRag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install langchain langchain-community chromadb sentence-transformers pypdf transformers huggingface_hub -q

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceHub

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

from google.colab import userdata

import os

In [41]:
# Configurar modelo de embeddings gratuito
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Definir modelo de LLM gratuito do Hugging Face
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('API_KEY')
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature": 0.7, "max_length": 500}
)


In [42]:
pdf_link = '/content/DOC-SF238339076816-20230503.pdf'

loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [31]:
len(pages)

31

In [43]:
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=200,
    length_function=len,
    add_start_index=True
)

In [44]:
store = InMemoryStore()
vectorStore = Chroma(embedding_function= embeddings_model, persist_directory='childVectorDB')

In [45]:
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorStore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)

parent_document_retriever.add_documents(pages, ids=None)

In [35]:
parent_document_retriever.vectorstore.get()

{'ids': ['2aa58b7e-4797-419f-a97b-a0a545124523',
  '81d05c9c-7e6e-4237-bf12-13ecbe30818f',
  '907be0c3-49f5-4153-b20b-8c2bbdd98d91',
  '80bab733-60ee-4c29-a90d-16d8d7f88ee6',
  'bf40d814-0e82-4ee5-970e-f8f4d40f474e',
  'baf8e885-d385-4375-886d-295c925cc0ff',
  '85fb48f4-109f-4bf3-82c3-c603f083a98f',
  '561c97c8-049f-4282-b037-990a0d5bef50',
  '2668e4a1-1bf9-4c43-a1d0-85e15a4ccb04',
  '5c7033a1-619f-4cd1-87e0-a45fe5a25bf6',
  '0491c312-fefa-4e8d-a73c-fd4b2315c45c',
  'df29662a-5e12-4c75-8105-7124f20f41c2',
  '5816da26-c155-40af-8b83-40909ba38268',
  'fd5231eb-903d-44f2-b5a2-20ff51df70bd',
  '21ca9544-0b34-4f11-b1a5-5eaeeb474f92',
  '700b769c-76c5-4de3-b6bf-2dc7de775098',
  '714c4fbc-da0b-4071-8618-43c57ade71f3',
  '742548a7-3502-4683-af04-a64da0611d4f',
  '85da30d8-f370-4641-9834-b63a073e655d',
  '4eb0b69f-7f68-4ef7-85a1-b008fab06743',
  'e2d69fef-2d63-419d-9bda-31183cbea646',
  'a5c5509e-a49e-43ac-b22c-b31010bc6c73',
  '41d8f245-cc8d-49f4-ba23-d5d40af930a3',
  'd3de072c-e3ca-4fcf-998e-

In [50]:
TEMPLATE = """
Você é um especialista em legislação e tecnologia. Use o contexto abaixo para responder à pergunta de forma objetiva e detalhada.

Pergunta:
{question}

Contexto:
{context}

Resposta:
"""


rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)


In [51]:
setup_retrival = RunnableParallel({'question': RunnablePassthrough(), 'context': parent_document_retriever})

output_parser = StrOutputParser()

In [52]:
parent_chain_retrival = setup_retrival | rag_prompt | llm | output_parser

In [53]:
parent_chain_retrival.invoke('Quais os principais riscos do marco legal de IA?')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


"Human:  \nVocê é um especialista em legislação e tecnologia. Use o contexto abaixo para responder à pergunta de forma objetiva e detalhada.\n\nPergunta:\nQuais os principais riscos do marco legal de IA?\n\nContexto:\n[Document(metadata={'producer': 'Aspose.Words for Java 23.3.0; modified using iText® 5.5.11 ©2000-2017 iText Group NV (AGPL-version)', 'creator': 'Microsoft Office Word', 'creationdate': '2023-05-03T18:22:00+00:00', 'author': 'fredfqd', 'moddate': '2023-05-03T19:15:38-03:00', 'source': '/content/DOC-SF238339076816-20230503.pdf', 'total_pages': 31, 'page': 8, 'page_label': '9', 'start_index': 0}, page_content='§ 4º Se o resultado da reclassificação identificar o sistema de \\ninteligência artificial como de alto risco, a realização de avaliação de impacto \\nalgorítmico e a adoção das demais medidas de governança previstas no \\nCapítulo IV serão obrigatórias, sem prejuízo de eventuais penalidades em caso \\nde avaliação preliminar fraudulenta, incompleta ou inverídica. \\

In [54]:
response = parent_chain_retrival.invoke('Quais os principais riscos do marco legal de IA?')
print(response)


Human:  
Você é um especialista em legislação e tecnologia. Use o contexto abaixo para responder à pergunta de forma objetiva e detalhada.

Pergunta:
Quais os principais riscos do marco legal de IA?

Contexto:
[Document(metadata={'producer': 'Aspose.Words for Java 23.3.0; modified using iText® 5.5.11 ©2000-2017 iText Group NV (AGPL-version)', 'creator': 'Microsoft Office Word', 'creationdate': '2023-05-03T18:22:00+00:00', 'author': 'fredfqd', 'moddate': '2023-05-03T19:15:38-03:00', 'source': '/content/DOC-SF238339076816-20230503.pdf', 'total_pages': 31, 'page': 8, 'page_label': '9', 'start_index': 0}, page_content='§ 4º Se o resultado da reclassificação identificar o sistema de \ninteligência artificial como de alto risco, a realização de avaliação de impacto \nalgorítmico e a adoção das demais medidas de governança previstas no \nCapítulo IV serão obrigatórias, sem prejuízo de eventuais penalidades em caso \nde avaliação preliminar fraudulenta, incompleta ou inverídica. \nSeção II \nR

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
